# Obtención de datos meteorológicos con la API de la AEMET

Constará de varias funciones para obtener los datos históricos a partir del 22 de junio de la API (en la zona de Cangas de Onís como municipio), y luego las previsiones para los siguientes días (en la zona de montaña de los Picos de Europa). Se construirán algunos dataframe con los que podremos construir un CSV y exportarlo. 

> NOTA: Es necesario proporcionar una API KEY para acceder a la AEMET.

Primero, vamos a agregar la ruta del directorio `keys` del proyecto al PATH de Python.

In [8]:
import sys
import os

In [9]:
# Ruta absoluta del directorio "keys"
base_dir = os.path.dirname(os.path.abspath(os.getcwd()))
ruta_keys = os.path.join(base_dir, "keys")

# Validación de que el directorio existe
if not os.path.isdir(ruta_keys):
    raise FileNotFoundError(f"El directorio 'keys' no existe en: {ruta_keys}")

# Validación de que el archivo existe
ruta_api_key = os.path.join(ruta_keys, "api_key.py")
if not os.path.isfile(ruta_api_key):
    raise FileNotFoundError(f"El archivo api_key.py no existe en: {ruta_keys}")

# Añadir el directorio al PATH
if ruta_keys not in sys.path:
    sys.path.append(ruta_keys)

In [10]:
# Paquetes
try:
    from api_key import api_key
except ImportError as e:
    raise ImportError(f"No se pudo importar api_key. Error: {str(e)}")

import requests
import json
import pandas as pd
from datetime import datetime, timedelta, timezone
import time

## Definición de constantes

In [78]:
querystring = {"api_key": api_key}
headers = {
	'cache-control': "no-cache"
}

ID_MUNICIPIO = '33012'  # Cangas de Onís
ZONA = 'peu1'  # Zona montañosa Picos de Europa
IDEMA = "1178R"  # Sotres
IDEMA2 = "1176"  # Cangas de Onís
IDEMA3 = "1175X"  # Bulnes
hoy = datetime.now(timezone.utc)

fecha_inicio = "2025-06-22T00:00:00UTC"
fecha_fin = "2025-06-29T00:00:00UTC"

fecha_inicio2 = "2025-07-01T00:00:00UTC"
fecha_fin2 = (hoy - timedelta(days=4)).strftime("%Y-%m-%dT00:00:00UTC")

## Obtención de valores climatológicos pasados

Esta función usa el endpoint de la AEMET para obtener los valores climatológicos por días observados en una determinada estación, dentro de un rango de días especificado. Devuelve el dataframe con las columnas de interés.

In [ ]:
def consulta_historicos(inicio: str, fin: str, estacion:str=IDEMA) -> pd.DataFrame | None:
	url = f"https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/{inicio}/fechafin/{fin}/estacion/{estacion}"

	# Si la información está ya almacenada, la leemos de un fichero
	if os.path.exists(os.path.join(base_dir, "data", "raw", f"datos_aemet_historicos_{inicio}.json")):
		with open(os.path.join(base_dir, "data", "raw", f"datos_aemet_historicos_{inicio}.json")) as f:
			prediccion = json.load(f)
	else:
		# Si no, hacemos la petición y guardamos la información devuelta
		response = requests.request("GET", url, headers=headers, params=querystring)

		if response.status_code == 200:

			# Descarga de los datos reales
			datos = response.json()
			url_datos = datos["datos"]
			# print(url_datos)
			response_datos = requests.request("GET", url_datos, headers=headers)

			if response_datos.status_code == 200:
				prediccion = response_datos.json()

				# Almacenar la información devuelta en un fichero
				with open(os.path.join(base_dir, "data", "raw", f"datos_aemet_historicos_{inicio}.json"), "w") as f:
					json.dump(prediccion, f)

	if prediccion:
		# Convertir a dataframe y poner la fecha en el índice
		df = pd.DataFrame(prediccion)
		if "fecha" in df.columns:
			df["fecha"] = pd.to_datetime(df["fecha"])

		# Eliminar columnas que no nos interesan
		df_nuevo = df.drop(columns=["indicativo", "nombre", "provincia", "altitud", "horatmin", "horatmax", "tmed", "horaracha", "horaPresMax", "horaPresMin", "horaHrMax", "horaHrMin"])
		
		return df_nuevo

In [ ]:
# Cambio en la configuración de pandas para mostrar el dataframe
pd.set_option('display.max_columns', None)      # Muestra todas las columnas
pd.set_option('display.width', None)            # Usa el ancho completo del terminal o Jupyter
pd.set_option('display.expand_frame_repr', False)  # Evita que pandas divida líneas

df_pasado = consulta_historicos(fecha_inicio, fecha_fin)
time.sleep(2)
df_pasado2 = consulta_historicos(fecha_inicio2, fecha_fin2)

# Concatenar ambos dataframes
df_pasado_completo = pd.concat([df_pasado, df_pasado2], ignore_index=True)
print(df_pasado_completo)

        fecha  prec  tmin  tmax dir velmedia racha presMax presMin hrMedia hrMax hrMin
0  2025-06-22   0,0  11,9  20,4  15      1,4   7,2   884,2   881,9      99   100    61
1  2025-06-23   2,0  11,7  25,0  24      2,5  16,1   884,0   882,2      64   100    40
2  2025-06-24  23,8  13,7  26,6  23      5,6  21,7   884,7   878,7      54    94    34
3  2025-06-25   2,0  11,5  19,9  12      3,6  14,7   882,5   875,5      86   100    50
4  2025-06-26   0,0  11,4  18,2  07      1,4   6,1   886,9   882,5      95   100    80
5  2025-06-27   0,0  14,8  23,5  06      2,5   7,5   889,2   886,2      58    88    51
6  2025-06-28   0,0  16,7  26,7  06      2,8   6,7   889,0   887,9      56    90    46
7  2025-06-29   6,0  14,6  28,9  26      1,4  22,2   888,0   884,5      43    85    28
8  2025-07-01   0,4  14,4  29,1  10      2,2   6,9   882,9   880,4      61   100    30
9  2025-07-02   5,8  11,4  16,0  08      3,3  10,0   885,9   879,9     100   100    94
10 2025-07-03  30,6  11,0  17,0  10      3,

Una vez tenemos estos datos, vamos a completarlos con la información obtenida en la observación directa y en las previsiones de esos días. Esto tendrá que hacerse manualmente. 
 
> NOTA: Si se quiere usar este programa para otros fines y se buscan otros datos u otras fechas, habrá que rellenar manualmente estas columnas o dejarlas vacías con comillas. La AEMET no almacena un histórico con estos datos, por lo que si el usuario no los ha guardado, no hay manera de acceder a ellos.

In [85]:
if df_pasado_completo is not None:
	# low/high/mid
	df_pasado_completo["altonubes"] = ["low", "", "mid", "", "low", "mid", "mid", "mid", "low", "low", "low", "low"]      

	# abundante/escasa/media
	df_pasado_completo["nubosidad"] = ["abundante", "","escasa", "abundante", "escasa", "escasa", "media", "escasa", "abundante", "abundante", "abundante", "abundante"] 

	# chubascos/posible/no
	df_pasado_completo["lluvia"] = ["posible", "", "chubascos", "chubascos", "no", "no", "no", "posible", "posible", "posible", "posible", "posible"]

	# SI/NO  (en base a las imágenes finales)
	df_pasado_completo["SUBIR"] = ["NO", "NO", "SI", "SI", "NO", "SI", "SI", "SI", "NO", "NO", "NO", "NO"]  

	print(df_pasado_completo)

        fecha  prec  tmin  tmax dir velmedia racha presMax presMin hrMedia hrMax hrMin altonubes  nubosidad     lluvia SUBIR
0  2025-06-22   0,0  11,9  20,4  15      1,4   7,2   884,2   881,9      99   100    61       low  abundante    posible    NO
1  2025-06-23   2,0  11,7  25,0  24      2,5  16,1   884,0   882,2      64   100    40                                    NO
2  2025-06-24  23,8  13,7  26,6  23      5,6  21,7   884,7   878,7      54    94    34       mid     escasa  chubascos    SI
3  2025-06-25   2,0  11,5  19,9  12      3,6  14,7   882,5   875,5      86   100    50            abundante  chubascos    SI
4  2025-06-26   0,0  11,4  18,2  07      1,4   6,1   886,9   882,5      95   100    80       low     escasa         no    NO
5  2025-06-27   0,0  14,8  23,5  06      2,5   7,5   889,2   886,2      58    88    51       mid     escasa         no    SI
6  2025-06-28   0,0  16,7  26,7  06      2,8   6,7   889,0   887,9      56    90    46       mid      media         no    SI


Una vez tenemos el dataframe completo, vamos a establecer como índice la fecha.

In [93]:
df_pasado_completo.set_index(df_pasado_completo.fecha, inplace=True)
df_pasado_final = df_pasado_completo.drop("fecha", axis=1)
df_pasado_final

,prec,tmin,tmax,dir,velmedia,racha,presMax,presMin,hrMedia,hrMax,hrMin,altonubes,nubosidad,lluvia,SUBIR
fecha,,,,,,,,,,,,,,,
2025-06-22,"0,0","11,9","20,4",15,"1,4","7,2","884,2","881,9",99,100,61,low,abundante,posible,NO
2025-06-23,"2,0","11,7","25,0",24,"2,5","16,1","884,0","882,2",64,100,40,,,,NO
2025-06-24,"23,8","13,7","26,6",23,"5,6","21,7","884,7","878,7",54,94,34,mid,escasa,chubascos,SI
2025-06-25,"2,0","11,5","19,9",12,"3,6","14,7","882,5","875,5",86,100,50,,abundante,chubascos,SI
2025-06-26,"0,0","11,4","18,2",07,"1,4","6,1","886,9","882,5",95,100,80,low,escasa,no,NO
2025-06-27,"0,0","14,8","23,5",06,"2,5","7,5","889,2","886,2",58,88,51,mid,escasa,no,SI
2025-06-28,"0,0","16,7","26,7",06,"2,8","6,7","889,0","887,9",56,90,46,mid,media,no,SI
2025-06-29,"6,0","14,6","28,9",26,"1,4","22,2","888,0","884,5",43,85,28,mid,escasa,posible,SI
2025-07-01,"0,4","14,4","29,1",10,"2,2","6,9","882,9","880,4",61,100,30,low,abundante,posible,NO


Por último, podemos guardar el dataframe construido en un CSV para procesarlo más tarde:

In [96]:
df_pasado_final.to_csv('data.csv', sep=";", encoding="utf-8")

## Obtención de previsiones en zona de montaña

Esta función imprime las previsiones de los próximos 4 días (desde hoy hasta dentro de 3 días). Como se dan de forma textual, esto servirá para crear manualmente el dataframe.

In [22]:
def consulta_mountain(zona=ZONA):
	url = f"https://opendata.aemet.es/opendata/api/prediccion/especifica/montaña/pasada/area/{zona}/dia/"
	dict_respuesta = dict()

	for i in range(4):
		response = requests.request("GET", url+str(i), headers=headers, params=querystring)

		if response.status_code == 200:

			# Descarga de los datos reales
			datos = response.json()
			url_datos = datos["datos"]
			print(url_datos)

			
			response_datos = requests.request("GET", url_datos, headers=headers)

			if response_datos.status_code == 200:
				prediccion = response_datos.json()

				# En base a la respuesta dada, rellenamos un JSON propio
			

In [23]:
consulta_mountain()

https://opendata.aemet.es/opendata/sh/ddf16566
https://opendata.aemet.es/opendata/sh/f886f646
https://opendata.aemet.es/opendata/sh/c8bf7fe4
https://opendata.aemet.es/opendata/sh/359a1f45
